In [2]:
import sys
import os

# Add your project root to sys.path (adjust the path below)
project_root = '/work/pi_shenoy_umass_edu/hshastri/FMaaS-motivation/timeseries/amazon-chronos/src'  # the folder containing 'momentfm'
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [4]:

from torch.utils.data import DataLoader
from momentfm.data.classification_dataset import ClassificationDataset
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA
from chronos import ChronosPipeline


In [5]:
train_dataset = ClassificationDataset(data_split='train')
test_dataset = ClassificationDataset(data_split='test')
print(test_dataset)

FileNotFoundError: [Errno 2] No such file or directory: '../data/ECG5000_TRAIN.ts'

In [ ]:


idx = np.random.randint(0, len(train_dataset))
heartbeat_start = np.argmax(train_dataset[idx][1])
heartbeat = train_dataset[idx][0].squeeze()[heartbeat_start:]
label = train_dataset[idx][2]
plt.plot(heartbeat, c='darkblue')
plt.title(f"idx={idx} | label={label}")
plt.show()

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,)

In [ ]:
def get_embedding(model, dataloader):
    embeddings, labels = [], []
    with torch.no_grad():
        for batch_x, batch_masks, batch_labels in tqdm(dataloader, total=len(dataloader)):
            batch_x = batch_x.to("cuda").float()
            batch_masks = batch_masks.to("cuda")
            embeddings, tokenizer_state = pipeline.embed(batch_x)
            # output = model(x_enc=batch_x, input_mask=batch_masks) # [batch_size x d_model (=1024)]

            # embedding = output.embeddings
            embedding = output.embeddings.mean(dim=1)  # pooled to shape (batch_size, 1024)

            embeddings.append(embedding.detach().cpu().numpy())
            labels.append(batch_labels)        

    embeddings, labels = np.concatenate(embeddings), np.concatenate(labels)
    return embeddings, labels

In [ ]:
model.to("cuda").float()

train_embeddings, train_labels = get_embedding(model, train_dataloader)
test_embeddings, test_labels = get_embedding(model, test_dataloader)

print(train_embeddings.shape, train_labels.shape)
print(test_embeddings.shape, test_labels.shape)

In [ ]:
test_embeddings_manifold = PCA(n_components=2).fit_transform(test_embeddings) 

plt.title(f"ECG5000 Test Embeddings", fontsize=20)
plt.scatter(
    test_embeddings_manifold[:, 0], 
    test_embeddings_manifold[:, 1],
    c=test_labels.squeeze(),
    cmap='magma'
)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.colorbar(
    boundaries=np.arange(
    min(test_labels),
    max(test_labels)+1, 1)
)
plt.show()
print(min(test_labels))
print(max(test_labels))